In [3]:
import webbrowser
import requests
import msal
import json

import os
os.chdir("/home/daniel/de_AI_Fabriek/belasting-agent-playground/")

from keys import O365_CLIENT_SECRET, O365_CLIENT_ID

SCOPES = ["User.Read"]

# AUTHORITY = 'https://login.microsoftonline.com/' + "b4790366-ebc1-458b-8baf-4aed4d12f0fc"
AUTHORITY = 'https://login.microsoftonline.com/common'

## msal

In [1]:
def generate_acces_token(app_id, scopes, authority):
    acces_token_cache = msal.SerializableTokenCache()
    
    # read token cache
    if os.path.exists("msg_api_token_access.json"):
        acces_token_cache.deserialize(open("msg_api_token_access.json", "r").read())
        
    client = msal.PublicClientApplication(
        app_id,
        authority=authority,
        token_cache=acces_token_cache
    )
    
    token_response = None
     
    accounts = client.get_accounts()
                  
    if accounts:
        
        # Assuming the end user chose this one to proceed
        chosen_account = accounts[0]    
        
        # Now let's try to find a token in cache for this account
        # or by using refresh token
        token_response = client.acquire_token_silent(scopes, account=chosen_account)

    if not token_response:
        flow = client.initiate_device_flow(scopes=scopes)
        print('user code: ', flow['user_code'])
        webbrowser.open(flow['verification_uri'])
        
        token_response = client.acquire_token_by_device_flow(flow)
        print(flow["message"])
    
    if "access_token" in token_response:
        print("Token was obtained from:", token_response["token_source"])
        if os.getenv('ENDPOINT'):
            # Calling a web API using the access token
            api_result = requests.get(
                os.getenv('ENDPOINT'),
                hearders={'Authorization': 'Bearer ' + token_response["access_token"]},
            ).json()    
            print("Web API call result", json.dumps(api_result, indent=2))
        else:
            print("Token acquisition result", json.dumps(token_response, indent=2))
    else:
        print("Token acquisition failed", token_response)
    
    # write to persistent cache
    with open("msg_api_token_access.json", "w") as _f:
        _f.write(acces_token_cache.serialize())
    return token_response
        
    

In [4]:
token_response = generate_acces_token(O365_CLIENT_ID, SCOPES, AUTHORITY)

Token was obtained from: identity_provider
Token acquisition result {
  "token_type": "Bearer",
  "scope": "openid profile User.Read email",
  "expires_in": 4362,
  "ext_expires_in": 4362,
  "access_token": "eyJ0eXAiOiJKV1QiLCJub25jZSI6ImlSR2ViNUdrV2FUS0wzbXJIM1hXSXl4clJJaHV2R3pfRkE3dWRNUXdzRzAiLCJhbGciOiJSUzI1NiIsIng1dCI6IllUY2VPNUlKeXlxUjZqekRTNWlBYnBlNDJKdyIsImtpZCI6IllUY2VPNUlKeXlxUjZqekRTNWlBYnBlNDJKdyJ9.eyJhdWQiOiIwMDAwMDAwMy0wMDAwLTAwMDAtYzAwMC0wMDAwMDAwMDAwMDAiLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC8wMDMwZTJmOC05Njg4LTQ5YzgtYWU0Zi1jZWY2N2E3ZWM4ZmYvIiwiaWF0IjoxNzM4NzA4NTY4LCJuYmYiOjE3Mzg3MDg1NjgsImV4cCI6MTczODcxMzIzMSwiYWNjdCI6MCwiYWNyIjoiMSIsImFpbyI6IkFUUUF5LzhaQUFBQXVvMGVPQlhCVUtYSDh1WEhQak9hMDRVK2RhZm9yQ3JyS2hEYllzbTg3K3hrNmN3Wmc3bEVJcElmekdQbWc3dmUiLCJhbXIiOlsicHdkIl0sImFwcF9kaXNwbGF5bmFtZSI6ImJlbGFzdGluZy1hYW5naWZ0ZW4tYWdlbnQiLCJhcHBpZCI6IjZmZTRjOTlhLTc4ODYtNDQ2My05MWJiLWNmMTMzYzVkZDE4NCIsImFwcGlkYWNyIjoiMCIsImZhbWlseV9uYW1lIjoidmFuIE9vc3Rlcm9vbSIsImdpdmVuX25hbWUiOiJEYW5

In [5]:
token_response


{'token_type': 'Bearer',
 'scope': 'openid profile User.Read email',
 'expires_in': 4362,
 'ext_expires_in': 4362,
 'access_token': 'eyJ0eXAiOiJKV1QiLCJub25jZSI6ImlSR2ViNUdrV2FUS0wzbXJIM1hXSXl4clJJaHV2R3pfRkE3dWRNUXdzRzAiLCJhbGciOiJSUzI1NiIsIng1dCI6IllUY2VPNUlKeXlxUjZqekRTNWlBYnBlNDJKdyIsImtpZCI6IllUY2VPNUlKeXlxUjZqekRTNWlBYnBlNDJKdyJ9.eyJhdWQiOiIwMDAwMDAwMy0wMDAwLTAwMDAtYzAwMC0wMDAwMDAwMDAwMDAiLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC8wMDMwZTJmOC05Njg4LTQ5YzgtYWU0Zi1jZWY2N2E3ZWM4ZmYvIiwiaWF0IjoxNzM4NzA4NTY4LCJuYmYiOjE3Mzg3MDg1NjgsImV4cCI6MTczODcxMzIzMSwiYWNjdCI6MCwiYWNyIjoiMSIsImFpbyI6IkFUUUF5LzhaQUFBQXVvMGVPQlhCVUtYSDh1WEhQak9hMDRVK2RhZm9yQ3JyS2hEYllzbTg3K3hrNmN3Wmc3bEVJcElmekdQbWc3dmUiLCJhbXIiOlsicHdkIl0sImFwcF9kaXNwbGF5bmFtZSI6ImJlbGFzdGluZy1hYW5naWZ0ZW4tYWdlbnQiLCJhcHBpZCI6IjZmZTRjOTlhLTc4ODYtNDQ2My05MWJiLWNmMTMzYzVkZDE4NCIsImFwcGlkYWNyIjoiMCIsImZhbWlseV9uYW1lIjoidmFuIE9vc3Rlcm9vbSIsImdpdmVuX25hbWUiOiJEYW5pZWwiLCJpZHR5cCI6InVzZXIiLCJpcGFkZHIiOiIyMDAxOjFjMDA6ZDA0OjIyMDA6ZmUyMDo4ND

In [9]:
import datetime

print(datetime.datetime.now())

print(datetime.datetime.now() + datetime.timedelta(0, 3855))

2025-02-04 11:09:41.222811
2025-02-04 12:13:56.223080


In [ ]:
import time
x = 0

while x < 10:
    time.sleep(1)
    print(datetime.datetime.now())
    x += 1

print(x)

